In [1]:
import autosklearn.classification
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics
import openml

import scipy as sp
import scipy.stats
import numpy as np
import pandas as pd
import itertools as it
from tqdm import tqdm_notebook as tqdm
import ast

/home/felix/anaconda3/lib/python3.7/site-packages/pyparsing.py:3190: FutureWarning: Possible set intersection at position 3
  self.re = re.compile(self.reString)


# Results

In [2]:
sparse_datasets = [1590, 1486, 4541,4534,4135,40978,40670, 1111,42732, 42733]
faulty_datasets = [23, 31, 188, 40996, 41161, 42734]

In [3]:
dfResults = pd.read_csv("icml2021.csv", delimiter=";")
dfResults = dfResults[~dfResults["openmlid"].isin(faulty_datasets)]
#dfResults = dfResults[dfResults["errorrate"].notna()]
dfResults

,experiment_id,openmlid,seed,algorithm,cpus,memory_max,time_created,host,executor,time_started,chosenmodel,errorrate,onlinedata,exception,time_end
0,4031,1485,0,auto-sklearn,8,24000,2021-01-19 22:53:26,node10-033,oculus-7036263-128,2021-01-20 13:26:17,"[(1.000000, SimpleClassificationPipeline({'bal...",0.088462,NaN,NaN,2021-01-20 14:26:32
1,4032,1485,1,auto-sklearn,8,24000,2021-01-19 22:53:26,node05-021,oculus-7036263-38,2021-01-20 13:26:16,"[(1.000000, SimpleClassificationPipeline({'bal...",0.107692,NaN,NaN,2021-01-20 14:26:44
2,4033,1590,0,auto-sklearn,8,24000,2021-01-19 22:53:26,node01-003,oculus-7118133-0,2021-02-22 17:54:54,"[(1.000000, SimpleClassificationPipeline({'bal...",0.160524,NaN,NaN,2021-02-22 18:55:20
3,4034,1590,1,auto-sklearn,8,24000,2021-01-19 22:53:26,node05-031,oculus-7118133-39,2021-02-22 17:54:55,"[(1.000000, SimpleClassificationPipeline({'bal...",0.165029,NaN,NaN,2021-02-22 18:55:23
4,4035,1485,2,auto-sklearn,8,24000,2021-01-19 22:53:26,node08-047,oculus-7036263-66,2021-01-20 13:26:16,"[(1.000000, SimpleClassificationPipeline({'bal...",0.069231,NaN,NaN,2021-01-20 14:26:40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18079,25463,1067,9,naive-python-meta-noniterative-monotone,8,24000,2021-03-01 18:37:19,node11-030,oculus-7138810-6,2021-03-01 21:45:50,"(None, None, <class 'sklearn.ensemble._forest....",0.156398,"{""history"":[[702,0.19228070175438594],[784,0.1...",NaN,2021-03-01 22:46:30
18080,25464,1049,9,naive-python-meta-noniterative-monotone,8,24000,2021-03-01 18:37:19,node11-037,oculus-7138810-18,2021-03-01 21:42:09,"(StandardScaler(), None, <class 'sklearn.svm._...",0.095890,"{""history"":[[559,0.19949238578680206],[620,0.1...",NaN,2021-03-01 22:33:13
18081,25465,40975,8,naive-python-meta-noniterative-monotone,8,24000,2021-03-01 18:37:19,node12-043,oculus-7138810-106,2021-03-01 21:41:53,NaN,NaN,NaN,\n\tError class: java.lang.IllegalArgumentExce...,2021-03-01 21:41:54
18082,25466,40975,9,naive-python-meta-noniterative,8,24000,2021-03-01 18:37:19,node10-032,oculus-7138810-149,2021-03-01 21:46:59,NaN,NaN,NaN,\n\tError class: java.lang.IllegalArgumentExce...,2021-03-01 21:46:59


In [4]:
def get_autosklearn_pipeline(desc):
    pattern = "SimpleClassificationPipeline("
    pipeline_json = ast.literal_eval(desc[desc.index(pattern) + len(pattern):desc.index("\n")])[0]
    return pipeline_json

def uses_feature_preprocessing(pipeline):
    for key in pipeline:
        if "feature_preprocessor:__choice__" in key and not pipeline[key] in ["no_preprocessing", "select_percentile_classification"]:
            return True
    return False

In [5]:
for openmlid, dfDataset in dfResults[dfResults["algorithm"].str.contains("auto-weka")].groupby("openmlid"):
    print(openmlid, len(dfDataset))

3 10
12 10
54 10
181 10
1049 10
1067 10
1111 10
1457 10
1461 10
1464 10
1468 10
1475 10
1485 10
1486 10
1487 10
1489 10
1494 10
1515 10
1590 10
4134 10
4135 10
4534 10
4538 10
4541 10
23512 10
23517 10
40498 10
40668 10
40670 10
40685 10
40701 10
40900 10
40975 10
40978 10
40981 10
40982 10
40983 10
40984 10
41027 10
41138 10
41142 10
41143 10
41144 10
41145 10
41146 10
41147 10
41150 10
41156 10
41157 10
41158 10
41159 10
41162 10
41163 10
41164 10
41165 10
41166 10
41167 10
41168 10
41169 10
42732 10
42733 10


In [6]:
def getResultTable(dfResults, rowAttribute, colAttribute, resultAttribute, col_labels = None, cellFormatter = None, comparativeFormatters = None, row_formatter = None):
    rows = []
    colnames = list(pd.unique(dfResults[colAttribute]))
    for rowIndex, dfRow in dfResults.groupby(rowAttribute):
        
        # identify all scores for row
        scores = {}
        for colName in colnames:
            if colName != rowAttribute:
                scoresForCell = dfRow[(dfRow[colAttribute] == colName) & (dfRow[resultAttribute].notna())][resultAttribute].values
                if len(scoresForCell) == 0:
                    scoresForCell = [np.nan]
                scores[colName] = sorted(scoresForCell)
        
        rows.append([rowIndex] + row_formatter(rowIndex, scores))
    
    with pd.option_context("max_colwidth", 1000):
        return pd.DataFrame(rows, columns=[rowAttribute] + [col_labels[col] if not col_labels is None and col in col_labels else col for col in colnames])

In [83]:
def get_default_best_formatting(scores, indices):
    cellFormatter = lambda x: str(np.round(scipy.stats.trim_mean(x, 0.1), 2)) + "$\pm$" + str(np.round(np.nanstd(x), 2)) if len([y for y in x if not np.isnan(y)]) > 0 else "nan"
    
    # get best score (and column name)
    bestMean = None
    bestCol = None
    for colName in scores:
        scoresForCell = scores[colName]
        meanScore = scipy.stats.trim_mean(scoresForCell, 0.1)
        if ~np.isnan(meanScore) and (bestCol is None or bestMean > meanScore):
            bestMean = meanScore
            bestCol = colName
    
    # now apply formatters to cells and add them to row
    bestScores = scores[bestCol] if not bestCol is None else []
    
    # formatting of best cells
    row = []
    if bestMean is None:
        for colName in indices:
            row.append("nan")
    else:
        for colName in indices:
            scoresForCell = scores[colName]
            curMean = scipy.stats.trim_mean(scoresForCell, 0.1)
            isBest = ((bestCol == colName) | (np.round(curMean, 2) == np.round(bestMean, 2)))
            if not isBest:
                n = min(len(bestScores), len(scoresForCell))
                if n > 0:
                    if n > 1:
                        wilcoxon = sp.stats.wilcoxon(bestScores[:n], scoresForCell[:n])
                        isSignificant = wilcoxon.pvalue < 0.05
                        isRelevant = np.round(np.abs(np.round(bestMean, 2) - np.round(curMean, 2)), 2) > 0.01
                    else:
                        isSignificant = False
                        isRelevant = False
                else:
                    isSignificant = True
                    isRelevant = True
            entry = cellFormatter(scoresForCell)
            if isBest:
                entry = "\\textbf{" + str(entry) + "}"
            elif entry != "nan" and not (isSignificant and isRelevant):
                entry = "\\underline{" + str(entry) + "}"
            row.append(entry)
    return row

def row_layouter(openmlid, scores):
    
    indices = [x for x in scores]
    row = get_default_best_formatting(scores, indices)
    
    # we now have one (formatted) entry for each column. Now add bullet/circ symbols
    naive_cols = [x for x in scores if "naive" in x]
    competitor_cols = [x for x in scores if not "naive" in x]
    for naive_col in naive_cols:
        scores_naive = scores[naive_col]
        mean_naive = np.round(scipy.stats.trim_mean(scores_naive, 0.1), 2)
        for competitor_col in competitor_cols:
            scores_competitor = scores[competitor_col]
            mean_competitor = np.round(scipy.stats.trim_mean(scores_competitor, 0.1), 2)
            n = min(len(scores_naive), len(scores_competitor))
            if n > 1 and scores_naive[:n] != scores_competitor[:n]:
                wilcoxon = sp.stats.wilcoxon(scores_naive[:n], scores_competitor[:n])
                isSignificant = wilcoxon.pvalue < 0.05
            else:
                isSignificant = False
            
            is_substantially_better = np.round(mean_naive - mean_competitor, 2) > 0.01 and isSignificant
            is_substantially_worse = np.round(mean_competitor - mean_naive, 2) > 0.01 and isSignificant
            i = indices.index(competitor_col)
            if is_substantially_worse:
                row[i] += " $\\circ$"
            elif is_substantially_better:
                if competitor_col == "auto-sklearn":
                    models = [get_autosklearn_pipeline(desc) for desc in dfResults[(dfResults["algorithm"] == "auto-sklearn") & (dfResults["openmlid"] == openmlid)]["chosenmodel"].values]
                    used_preprocessing = np.round(np.mean([uses_feature_preprocessing(mod) for mod in models]))
                    if used_preprocessing:
                        row[i] += " $\\dagger$"
                    else:
                        row[i] += " $\\bullet$"
                else:
                    row[i] += " $\\bullet$"
                
            else:
                row[i] += " \\phantom{$\\circ$}"
        
    return row


filter_python = dfResults["algorithm"] == "auto-sklearn"
filter_python = filter_python | ((dfResults["algorithm"].str.contains("python")) & (dfResults["algorithm"].str.contains("primitive")) & (dfResults["algorithm"].str.contains("noniterative")))
filter_python = filter_python | ((dfResults["algorithm"].str.contains("python")) & (dfResults["algorithm"].str.contains("validation")) & (dfResults["algorithm"].str.contains("noniterative")) & dfResults["algorithm"].str.contains("monotone"))

filter_java = dfResults["algorithm"] == "auto-weka"
filter_java = filter_java | (dfResults["algorithm"] == "mlplan")
filter_java = filter_java | ((dfResults["algorithm"].str.contains("java")) & (dfResults["algorithm"].str.contains("primitive")) & (dfResults["algorithm"].str.contains("noniterative")))
filter_java = filter_java | ((dfResults["algorithm"].str.contains("java")) & (dfResults["algorithm"].str.contains("validation")) & (dfResults["algorithm"].str.contains("monotone")))

col_labels_python = {
    "auto-sklearn": "asklearn",
    "naive-python-primitive-noniterative": "primitive",
    "naive-python-scaling-noniterative": "scaling",
    "naive-python-filtering-noniterative": "filtering",
    "naive-python-filtering-noniterative-monotone": "filtering",
    "naive-python-meta-noniterative": "meta",
    "naive-python-meta-noniterative-monotone": "meta",
    "naive-python-tuning-noniterative": "tuning",
    "naive-python-tuning-noniterative-monotone": "tuning",
    "naive-python-validation-noniterative": "validation",
    "naive-python-validation-noniterative-monotone": "full",
    "naive-java-primitive-noniterative": "primitive",
    "naive-java-filtering-noniterative": "filtering",
    "naive-java-filtering-noniterative-monotone": "filtering",
    "naive-java-meta-noniterative": "meta",
    "naive-java-meta-noniterative-monotone": "meta",
    "naive-java-tuning-noniterative": "tuning",
    "naive-java-tuning-noniterative-monotone": "tuning",
    "naive-java-validation-noniterative-monotone": "full",
    "naive-java-primitive-noniterative": "primitive",
    "naive-java-validation-noniterative": "validation",
    "naive-java-validation-noniterative-monotone": "full",
}


dfResultsFinalJava = getResultTable(dfResults[filter_java], "openmlid", "algorithm", "errorrate", col_labels=col_labels_python, row_formatter=row_layouter)[["openmlid", "auto-weka", "mlplan", "primitive", "full"]]

dfResultsFinalPython = getResultTable(dfResults[filter_python], "openmlid", "algorithm", "errorrate", col_labels=col_labels_python, row_formatter=row_layouter)

latex_colname_map = {
    "primitive_x": "primitive",
    "primitive_y": "primitive",
    "full_x": "full",
    "full_y": "full",
    "openmlid": "id"
}
with pd.option_context("max_colwidth", 1000):
    dfMerge = dfResultsFinalJava.merge(dfResultsFinalPython, on="openmlid").rename(columns=latex_colname_map)
    latex_colname_map_centered = {}
    for col in dfMerge.columns:
        latex_colname_map_centered[col] = "\\multicolumn{1}{c}{" + col + "}"
    dfMerge = dfMerge.rename(columns=latex_colname_map_centered)
    str_base_table = dfMerge.to_latex(index=False, escape=False)
    print(str_base_table.replace("{rlllllll}\n\\toprule", "{r||rr|rr||r|rr}\n\\toprule\n & \multicolumn{4}{c||}{WEKA backend} & \multicolumn{3}{c}{scikit-learn backend}\\\\"))
#dfResultsFinalJava.iloc[-10:]

/home/felix/anaconda3/lib/python3.7/site-packages/scipy/stats/morestats.py:2958: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/felix/anaconda3/lib/python3.7/site-packages/scipy/stats/morestats.py:2972: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")
/home/felix/anaconda3/lib/python3.7/site-packages/scipy/stats/morestats.py:2958: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/felix/anaconda3/lib/python3.7/site-packages/scipy/stats/morestats.py:2972: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


\begin{tabular}{r||rr|rr||r|rr}
\toprule
 & \multicolumn{4}{c||}{WEKA backend} & \multicolumn{3}{c}{scikit-learn backend}\\
 \multicolumn{1}{c}{id} &                                  \multicolumn{1}{c}{auto-weka} &                                     \multicolumn{1}{c}{mlplan} & \multicolumn{1}{c}{primitive} &   \multicolumn{1}{c}{full} &                                   \multicolumn{1}{c}{asklearn} & \multicolumn{1}{c}{primitive} &   \multicolumn{1}{c}{full} \\
\midrule
                      3 &       \textbf{0.0$\pm$0.0} \phantom{$\circ$} \phantom{$\circ$} &  \underline{0.01$\pm$0.01} \phantom{$\circ$} \phantom{$\circ$} &          \textbf{0.0$\pm$0.0} &       \textbf{0.0$\pm$0.0} &       \textbf{0.0$\pm$0.0} \phantom{$\circ$} \phantom{$\circ$} &          \textbf{0.0$\pm$0.0} &       \textbf{0.0$\pm$0.0} \\
                     12 &      \textbf{0.02$\pm$0.0} \phantom{$\circ$} \phantom{$\circ$} &              0.04$\pm$0.01 \phantom{$\circ$} \phantom{$\circ$} &     \underline{0.03$\pm

In [17]:
def getGameTable4Isolated(dfResults, primitive_column, col_labels):
    algorithms = [a for a in pd.unique(dfResults["algorithm"]) if a != primitive_column and not "monotone" in a]
    wins = {a: 0 for a in algorithms}
    wins_unique = {a: 0 for a in algorithms}
    losses = {a: 0 for a in algorithms}
    draws = {a: 0 for a in algorithms}
    for openmlid, dfReduced in dfResults.groupby("openmlid"):
        
        base_performance = scipy.stats.trim_mean(dfReduced[dfReduced["algorithm"] == primitive_column]["errorrate"].values, 0.1)
        for algorithm in algorithms:
            performance = scipy.stats.trim_mean(dfReduced[dfReduced["algorithm"] == algorithm]["errorrate"].values, 0.1)
            if performance <= base_performance - 0.01:
                wins[algorithm] += 1
                
                best_performance_of_other = min([scipy.stats.trim_mean(dfReduced[dfReduced["algorithm"] == a]["errorrate"].values, 0.1) for a in algorithms if a != algorithm])
                if best_performance_of_other > performance:
                    wins_unique[algorithm] += 1
                
            elif performance - 0.01 >= base_performance:
                losses[algorithm] += 1
            else:
                draws[algorithm] += 1
    
    # create data frame
    cols = []
    for algo in algorithms:
        cols.append([wins[algo], wins_unique[algo], losses[algo], draws[algo]])
    return pd.DataFrame(np.array(cols).T, columns=[col_labels[a] if a in col_labels else a for a in algorithms], index=["wins", "unique wins", "losses", "draws"])
    
filter_java = (dfResults["algorithm"].str.contains("java")) & (dfResults["algorithm"].str.contains("noniterative") & (~dfResults["algorithm"].str.contains("monotone")))

filter_python = (dfResults["algorithm"].str.contains("python")) & (dfResults["algorithm"].str.contains("noniterative") & (~dfResults["algorithm"].str.contains("monotone")))
filter_python = filter_python & ~dfResults["algorithm"].str.contains("wrapping")
col_labels = {
    "naive-python-primitive-noniterative": "primitive",
    "naive-python-scaling-noniterative": "scaling",
    "naive-python-filtering-noniterative": "filtering",
    "naive-python-meta-noniterative": "meta",
    "naive-python-tuning-noniterative": "tuning",
    "naive-python-validation-noniterative": "validation",
    "naive-java-primitive-noniterative": "primitive",
    "naive-java-scaling-noniterative": "scaling",
    "naive-java-filtering-noniterative": "filtering",
    "naive-java-meta-noniterative": "meta",
    "naive-java-tuning-noniterative": "tuning",
    "naive-java-validation-noniterative": "validation"
}

dfTournamentPython = getGameTable4Isolated(dfResults[filter_python], primitive_column="naive-python-primitive-noniterative", col_labels = col_labels)[["scaling", "filtering", "meta", "tuning", "validation"]]
dfTournamentJava = getGameTable4Isolated(dfResults[filter_java], primitive_column="naive-java-primitive-noniterative", col_labels = col_labels)[["filtering", "meta", "tuning", "validation"]]
tournament_latex = pd.concat([dfTournamentJava, dfTournamentPython], axis=1).to_latex()
print(tournament_latex.replace("{lrrrrrrrrr}\n\\toprule", "{l|rrrr||rrrrr}\n\\toprule\n & \multicolumn{4}{c||}{WEKA backend} & \multicolumn{5}{c}{scikit-learn backend}\\\\"))

\begin{tabular}{l|rrrr||rrrrr}
\toprule
 & \multicolumn{4}{c||}{WEKA backend} & \multicolumn{5}{c}{scikit-learn backend}\\
{} &  filtering &  meta &  tuning &  validation &  scaling &  filtering &  meta &  tuning &  validation \\
\midrule
wins        &          6 &     4 &       1 &           4 &        7 &          2 &     2 &       3 &           1 \\
unique wins &          5 &     1 &       0 &           3 &        7 &          2 &     0 &       0 &           1 \\
losses      &          2 &     5 &       4 &           4 &        1 &          1 &     1 &       2 &           1 \\
draws       &         53 &    52 &      56 &          53 &       53 &         58 &    58 &      56 &          59 \\
\bottomrule
\end{tabular}



## Monotone Applications

### Synergentic Gains Due to Combined Stages

In [21]:
def getSynnergyTable(dfResults, col_labels):
    algorithms = [a for a in pd.unique(dfResults["algorithm"]) if "monotone" in a]
    wins = {a: 0 for a in algorithms}
    losses = {a: 0 for a in algorithms}
    draws = {a: 0 for a in algorithms}
    for openmlid, dfReduced in dfResults.groupby("openmlid"):
        for algorithm in algorithms:
            
            base_algorithms = ["naive-python-primitive-noniterative", "naive-python-scaling-noniterative", algorithm.replace("-monotone", "")]
            if "naive-python-validation-noniterative" in base_algorithms:
                base_algorithms.append("naive-python-tuning-noniterative")
            if "naive-python-tuning-noniterative" in base_algorithms:
                base_algorithms.append("naive-python-filtering-noniterative")
            
            
            base_performances = []            
            for base_algorithm in base_algorithms:
                base_performances.append(scipy.stats.trim_mean(dfReduced[dfReduced["algorithm"] == base_algorithm]["errorrate"].values, 0.1))
            base_performance = min(base_performances)
            
            performance = scipy.stats.trim_mean(dfReduced[dfReduced["algorithm"] == algorithm]["errorrate"].values, 0.1)
            if performance <= base_performance - 0.01:
                wins[algorithm] += 1
            elif performance - 0.01 >= base_performance:
                losses[algorithm] += 1
                print("Loss for " + algorithm + " on" + str(openmlid))
            else:
                draws[algorithm] += 1
    
    # create data frame
    cols = []
    for algo in algorithms:
        cols.append([wins[algo], losses[algo], draws[algo]])
    return pd.DataFrame(np.array(cols).T, columns=[col_labels[a] if a in col_labels else a for a in algorithms], index=["wins", "losses", "draws"])
    
filter_python = ((dfResults["algorithm"].str.contains("python")) & (dfResults["algorithm"].str.contains("noniterative")))
filter_python = filter_python & ~dfResults["algorithm"].str.contains("wrapping")
filter_java = ((dfResults["algorithm"].str.contains("java")) & (dfResults["algorithm"].str.contains("noniterative")))
col_labels = {
    "naive-python-filtering-noniterative-monotone": "filtering",
    "naive-python-meta-noniterative-monotone": "meta",
    "naive-python-tuning-noniterative-monotone": "tuning",
    "naive-python-validation-noniterative-monotone": "validation",
    "naive-java-meta-noniterative-monotone": "meta",
    "naive-java-tuning-noniterative-monotone": "tuning",
    "naive-java-validation-noniterative-monotone": "validation"
}

dfSynergyPython = getSynnergyTable(dfResults[filter_python], col_labels = col_labels)[["filtering", "meta", "tuning", "validation"]]
dfSynergyJava = getSynnergyTable(dfResults[filter_java], col_labels = col_labels)[["meta", "tuning", "validation"]]
tournament_latex = pd.concat([dfSynergyJava, dfSynergyPython], axis=1).to_latex()
print(tournament_latex.replace("{lrrrrrrr}\n\\toprule", "{l||rrr|rrrr}\n\\toprule\n & \multicolumn{3}{c|}{WEKA backend} & \multicolumn{3}{c}{scikit-learn backend}\\\\"))

Loss for naive-python-validation-noniterative-monotone on12
Loss for naive-python-tuning-noniterative-monotone on54
Loss for naive-python-validation-noniterative-monotone on54
Loss for naive-python-tuning-noniterative-monotone on1049
Loss for naive-python-filtering-noniterative-monotone on1067
Loss for naive-python-validation-noniterative-monotone on1457
Loss for naive-python-meta-noniterative-monotone on1468
Loss for naive-python-meta-noniterative-monotone on1515
Loss for naive-python-tuning-noniterative-monotone on4134
Loss for naive-python-tuning-noniterative-monotone on4538
Loss for naive-python-validation-noniterative-monotone on4538
Loss for naive-python-validation-noniterative-monotone on40981
Loss for naive-python-validation-noniterative-monotone on40982
Loss for naive-python-tuning-noniterative-monotone on40984
Loss for naive-python-filtering-noniterative-monotone on41157
Loss for naive-python-tuning-noniterative-monotone on41157
Loss for naive-python-validation-noniterative-m

# APPENDIX

# Dataset Descriptions

In [102]:
datasets = [int(i) for i in pd.unique(dfResults["openmlid"])]
rows = []
for openmlid in datasets:
    dataset = openml.datasets.get_dataset(openmlid)
    row = [openmlid, dataset.name, int(dataset.qualities["NumberOfInstances"]), len(dataset.features), int(dataset.qualities["NumberOfClasses"])]
    rows.append(row)
    print(row)

DEBUG:openml.datasets.dataset:Data pickle file already exists and is up to date.
DEBUG:openml.datasets.dataset:Data pickle file already exists and is up to date.
DEBUG:openml.datasets.dataset:Data pickle file already exists and is up to date.


[1485, 'madelon', 2600, 501, 2]
[1590, 'adult', 48842, 15, 2]
[1515, 'micro-mass', 571, 1301, 20]


DEBUG:openml.datasets.dataset:Data pickle file already exists and is up to date.
DEBUG:openml.datasets.dataset:Data pickle file already exists and is up to date.


[1457, 'amazon-commerce-reviews', 1500, 10001, 50]
[1475, 'first-order-theorem-proving', 6118, 52, 6]


DEBUG:openml.datasets.dataset:Saved dataset 1468: cnae-9 to file /home/felix/.openml/cache/org/openml/www/datasets/1468/dataset.pkl.py3


...

[1067, 'kc1', 2109, 22, 2]


DEBUG:openml.datasets.dataset:Saved dataset 1049: pc4 to file /home/felix/.openml/cache/org/openml/www/datasets/1049/dataset.pkl.py3


[1049, 'pc4', 1458, 38, 2]


DEBUG:openml.datasets.dataset:Saved dataset 40975: car to file /home/felix/.openml/cache/org/openml/www/datasets/40975/dataset.pkl.py3


[40975, 'car', 1728, 7, 4]


In [108]:
dfDatasets = pd.DataFrame(rows, columns=["openmlid", "name", "instances", "features", "classes"]).sort_values("openmlid")
print(dfDatasets.to_latex(index=False))

\begin{tabular}{rlrrr}
\toprule
 openmlid &                                    name &  instances &  features &  classes \\
\midrule
        3 &                                kr-vs-kp &       3196 &        37 &        2 \\
       12 &                           mfeat-factors &       2000 &       217 &       10 \\
       54 &                                 vehicle &        846 &        19 &        4 \\
      181 &                                   yeast &       1484 &         9 &       10 \\
     1049 &                                     pc4 &       1458 &        38 &        2 \\
     1067 &                                     kc1 &       2109 &        22 &        2 \\
     1111 &                      KDDCup09\_appetency &      50000 &       231 &        2 \\
     1457 &                 amazon-commerce-reviews &       1500 &     10001 &       50 \\
     1461 &                          bank-marketing &      45211 &        17 &        2 \\
     1464 &        blood-transfusion-service-cen

# Detailed Results

## Isolated

### scikit-learn

In [84]:
filter_python = ((dfResults["algorithm"].str.contains("python")) & (dfResults["algorithm"].str.contains("noniterative")))
filter_python = filter_python & ~dfResults["algorithm"].str.contains("wrapping")
filter_python = filter_python | (dfResults["algorithm"] == "auto-sklearn")
filter_monotone = (~dfResults["algorithm"].str.contains("monotone"))
filter_results = (dfResults["algorithm"] == "auto-sklearn")
filter_results = filter_results | (filter_python & filter_monotone)
filter_results = filter_results | (filter_python & (dfResults["algorithm"].str.contains("primitive")))
filter_results = filter_results | (filter_python & (dfResults["algorithm"].str.contains("scaling")))

dfResultsMonotone = getResultTable(dfResults[filter_results], "openmlid", "algorithm", "errorrate", col_labels=col_labels_python, row_formatter=row_layouter)[["openmlid", "asklearn", "primitive", "scaling", "filtering", "meta", "tuning", "validation"]]
with pd.option_context("max_colwidth", 1000):
    print(dfResultsMonotone.to_latex(index=False, escape=False).replace("{rlllllll}", "{rrrrrrrr}"))

/home/felix/anaconda3/lib/python3.7/site-packages/scipy/stats/morestats.py:2958: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/felix/anaconda3/lib/python3.7/site-packages/scipy/stats/morestats.py:2972: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


\begin{tabular}{rrrrrrrr}
\toprule
 openmlid &                                                                                                                               asklearn &                  primitive &                    scaling &                  filtering &                       meta &                     tuning &                 validation \\
\midrule
        3 &       \textbf{0.0$\pm$0.0} \phantom{$\circ$} \phantom{$\circ$} \phantom{$\circ$} \phantom{$\circ$} \phantom{$\circ$} \phantom{$\circ$} &       \textbf{0.0$\pm$0.0} &       \textbf{0.0$\pm$0.0} &       \textbf{0.0$\pm$0.0} &                        nan &       \textbf{0.0$\pm$0.0} &      \textbf{0.0$\pm$0.01} \\
       12 &  \underline{0.03$\pm$0.02} \phantom{$\circ$} \phantom{$\circ$} \phantom{$\circ$} \phantom{$\circ$} \phantom{$\circ$} \phantom{$\circ$} &     \textbf{0.02$\pm$0.02} &     \textbf{0.02$\pm$0.01} &     \textbf{0.02$\pm$0.02} &     \textbf{0.02$\pm$0.02} &     \textbf{0.02$\pm$0.01} &     \textbf{0.

### weka

In [86]:
filter_weka = ((dfResults["algorithm"].str.contains("java")) & (dfResults["algorithm"].str.contains("noniterative")))
filter_weka = filter_weka & ~dfResults["algorithm"].str.contains("wrapping")
filter_monotone = ~(dfResults["algorithm"].str.contains("monotone"))
filter_results = (dfResults["algorithm"] == "auto-weka") | (dfResults["algorithm"] == "mlplan")
filter_results = filter_results | (filter_weka & filter_monotone)

dfResultsMonotone = getResultTable(dfResults[filter_results], "openmlid", "algorithm", "errorrate", col_labels=col_labels_python, row_formatter=row_layouter)[["openmlid", "auto-weka", "mlplan", "primitive", "filtering", "meta", "tuning", "validation"]]
with pd.option_context("max_colwidth", 1000):
    print(dfResultsMonotone.to_latex(index=False, escape=False).replace("{rlllllll}", "{rrrrrrrr}"))

/home/felix/anaconda3/lib/python3.7/site-packages/scipy/stats/morestats.py:2958: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/felix/anaconda3/lib/python3.7/site-packages/scipy/stats/morestats.py:2972: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


\begin{tabular}{rrrrrrrr}
\toprule
 openmlid &                                                                                                            auto-weka &                                                                                                               mlplan &                  primitive &                  filtering &                       meta &                     tuning &                 validation \\
\midrule
        3 &       \textbf{0.0$\pm$0.0} \phantom{$\circ$} \phantom{$\circ$} \phantom{$\circ$} \phantom{$\circ$} \phantom{$\circ$} &  \underline{0.01$\pm$0.01} \phantom{$\circ$} \phantom{$\circ$} \phantom{$\circ$} \phantom{$\circ$} \phantom{$\circ$} &       \textbf{0.0$\pm$0.0} &       \textbf{0.0$\pm$0.0} &       \textbf{0.0$\pm$0.0} &       \textbf{0.0$\pm$0.0} &       \textbf{0.0$\pm$0.0} \\
       12 &      \textbf{0.02$\pm$0.0} \phantom{$\circ$} \phantom{$\circ$} \phantom{$\circ$} \phantom{$\circ$} \phantom{$\circ$} &              0.04$\pm$0.01 \phant

## monotone

### scikit-learn

In [76]:
filter_python = ((dfResults["algorithm"].str.contains("python")) & (dfResults["algorithm"].str.contains("noniterative")))
filter_python = filter_python & ~dfResults["algorithm"].str.contains("wrapping")
filter_python = filter_python | (dfResults["algorithm"] == "auto-sklearn")
filter_monotone = (dfResults["algorithm"].str.contains("monotone"))
filter_results = (dfResults["algorithm"] == "auto-sklearn")
filter_results = filter_results | (filter_python & filter_monotone)
filter_results = filter_results | (filter_python & (dfResults["algorithm"].str.contains("primitive")))
filter_results = filter_results | (filter_python & (dfResults["algorithm"].str.contains("scaling")))

dfResultsMonotone = getResultTable(dfResults[filter_results], "openmlid", "algorithm", "errorrate", col_labels=col_labels_python, row_formatter=row_layouter)[["openmlid", "asklearn", "primitive", "scaling", "filtering", "meta", "tuning", "full"]]
with pd.option_context("max_colwidth", 1000):
    print(dfResultsMonotone.to_latex(index=False, escape=False).replace("{rlllllll}", "{rrrrrrrr}"))

/home/felix/anaconda3/lib/python3.7/site-packages/scipy/stats/morestats.py:2958: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/felix/anaconda3/lib/python3.7/site-packages/scipy/stats/morestats.py:2972: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


\begin{tabular}{rrrrrrrr}
\toprule
 openmlid &                                                                                                                               asklearn &                  primitive &                    scaling &                  filtering &                       meta &                     tuning &                       full \\
\midrule
        3 &       \textbf{0.0$\pm$0.0} \phantom{$\circ$} \phantom{$\circ$} \phantom{$\circ$} \phantom{$\circ$} \phantom{$\circ$} \phantom{$\circ$} &       \textbf{0.0$\pm$0.0} &       \textbf{0.0$\pm$0.0} &       \textbf{0.0$\pm$0.0} &                        nan &       \textbf{0.0$\pm$0.0} &       \textbf{0.0$\pm$0.0} \\
       12 &  \underline{0.03$\pm$0.02} \phantom{$\circ$} \phantom{$\circ$} \phantom{$\circ$} \phantom{$\circ$} \phantom{$\circ$} \phantom{$\circ$} &     \textbf{0.02$\pm$0.02} &     \textbf{0.02$\pm$0.01} &     \textbf{0.02$\pm$0.01} &     \textbf{0.02$\pm$0.02} &     \textbf{0.02$\pm$0.01} &  \underline{0.

### WEKA

In [79]:
filter_weka = ((dfResults["algorithm"].str.contains("java")) & (dfResults["algorithm"].str.contains("noniterative")))
filter_weka = filter_weka & ~dfResults["algorithm"].str.contains("wrapping")
filter_monotone = (dfResults["algorithm"].str.contains("monotone"))
filter_results = (dfResults["algorithm"] == "auto-weka") | (dfResults["algorithm"] == "mlplan")
filter_results = filter_results | (filter_weka & filter_monotone)
filter_results = filter_results | (filter_weka & (dfResults["algorithm"].str.contains("primitive")))
filter_results = filter_results | (filter_weka & (dfResults["algorithm"].str.contains("filtering")))

dfResultsMonotone = getResultTable(dfResults[filter_results], "openmlid", "algorithm", "errorrate", col_labels=col_labels_python, row_formatter=row_layouter)[["openmlid", "auto-weka", "mlplan", "primitive", "filtering", "meta", "tuning", "full"]]
with pd.option_context("max_colwidth", 1000):
    print(dfResultsMonotone.to_latex(index=False, escape=False).replace("{rlllllll}", "{rrrrrrrr}"))

/home/felix/anaconda3/lib/python3.7/site-packages/scipy/stats/morestats.py:2958: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/felix/anaconda3/lib/python3.7/site-packages/scipy/stats/morestats.py:2972: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


\begin{tabular}{rrrrrrrr}
\toprule
 openmlid &                                                                                                            auto-weka &                                                                                                               mlplan &                  primitive &                  filtering &                       meta &                     tuning &                       full \\
\midrule
        3 &       \textbf{0.0$\pm$0.0} \phantom{$\circ$} \phantom{$\circ$} \phantom{$\circ$} \phantom{$\circ$} \phantom{$\circ$} &  \underline{0.01$\pm$0.01} \phantom{$\circ$} \phantom{$\circ$} \phantom{$\circ$} \phantom{$\circ$} \phantom{$\circ$} &       \textbf{0.0$\pm$0.0} &       \textbf{0.0$\pm$0.0} &       \textbf{0.0$\pm$0.0} &      \textbf{0.0$\pm$0.01} &       \textbf{0.0$\pm$0.0} \\
       12 &      \textbf{0.02$\pm$0.0} \phantom{$\circ$} \phantom{$\circ$} \phantom{$\circ$} \phantom{$\circ$} \phantom{$\circ$} &                        0.04$\pm$0

# Runtimes Per Stage

In [52]:
def getPhaseRuntimeDF(df):
    algorithms = pd.unique(df["algorithm"])
    stages = ["probing", "scaling", "filtering", "meta", "tuning", "validation"]
    if len(algorithms) != 1:
        raise Exception("The dataframe must contain entries for EXACTLY one algorithm")
    algorithm = algorithms[0]
    rows = []
    for openmlid, dfDataset in df.groupby("openmlid"):
        stage_times = {}
        for seed, dfSeed in dfDataset.groupby("seed"):
            try:
                json = ast.literal_eval(dfSeed["onlinedata"].values[0])
                for stage in json["stageruntimes"]:
                    if not stage in stage_times:
                        stage_times[stage] = []
                    if str(json["stageruntimes"][stage]) != "nan":
                        stage_times[stage].append(json["stageruntimes"][stage])
            except ValueError:
                print("IGNORING ENTRY")
                pass
        
        row = [openmlid]
        total = 0
        for stage in stages:
            if stage in stage_times:
                stage_contribution = min(3600, int(np.round(scipy.stats.trim_mean(stage_times[stage], 0.1) / 1000)))
                stage_contribution = min(3600 - total, stage_contribution)
                row.append(stage_contribution)
                total += stage_contribution
            else:
                row.append(0)
        row.append(total)
        rows.append(row)
    return pd.DataFrame(rows, columns=["openmlid"] + stages + ["Total"])
        #print(openmlid, len(dfDataset))

## WEKA

In [53]:
stageTimeTable = getPhaseRuntimeDF(dfResults.query("algorithm == 'naive-java-validation-noniterative-monotone'"))
print(stageTimeTable.to_latex(index=False))

IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
\begin{tabular}{rrrrrrrr}
\toprule
 openmlid &  probing &  scaling &  filtering &  meta &  tuning &  validation &  Total \\
\midrule
        3 &      123 &        0 &         65 &    80 &      28 &         127 &    423 \\
       12 &      558 &        0 &        547 &    33 &      58 &         168 &   1364 \\
       54 &       19 &        0 &         17 &    26 &      22 &          42 &    126 \\
      181 &       27 &        0 &          6 &    11 &      15 &          26 &     85 \\
     1049 &       62 &        0 &         44 &     8 &      21 &          41 &    176 \\
     1067 &       53 &        0 &         34 &    21 &      19 &          42 &    169 \\
     1111 &     3600 &        0 &          0 &     0 &       0 &           0 &   3600 \\
     1457 &     1398 &        0 &       1085 &    50 &      10 &

## scikit-learn

In [54]:
stageTimeTable = getPhaseRuntimeDF(dfResults.query("algorithm == 'naive-python-validation-noniterative-monotone'"))
print(stageTimeTable.to_latex(index=False))

IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING ENTRY
IGNORING E

# Other Stuff

In [58]:
ds = openml.datasets.get_dataset(1515)
df = ds.get_data()[0]
print(df)
y = df[ds.default_target_attribute].values
X = df[[c for c in df.columns if c != ds.default_target_attribute]].values
timeout = 60 * 5

DEBUG:openml.datasets.dataset:Data pickle file already exists and is up to date.


      V1   V2           V3   V4   V5   V6            V7   V8   V9  V10  ...  \
0    0.0  0.0       0.0000  0.0  0.0  0.0  44431.921875  0.0  0.0  0.0  ...   
1    0.0  0.0       0.0000  0.0  0.0  0.0      0.000000  0.0  0.0  0.0  ...   
2    0.0  0.0       0.0000  0.0  0.0  0.0      0.000000  0.0  0.0  0.0  ...   
3    0.0  0.0       0.0000  0.0  0.0  0.0      0.000000  0.0  0.0  0.0  ...   
4    0.0  0.0       0.0000  0.0  0.0  0.0   4701.712402  0.0  0.0  0.0  ...   
..   ...  ...          ...  ...  ...  ...           ...  ...  ...  ...  ...   
566  0.0  0.0       0.0000  0.0  0.0  0.0      0.000000  0.0  0.0  0.0  ...   
567  0.0  0.0       0.0000  0.0  0.0  0.0      0.000000  0.0  0.0  0.0  ...   
568  0.0  0.0       0.0000  0.0  0.0  0.0      0.000000  0.0  0.0  0.0  ...   
569  0.0  0.0  145578.5625  0.0  0.0  0.0      0.000000  0.0  0.0  0.0  ...   
570  0.0  0.0  626135.8125  0.0  0.0  0.0      0.000000  0.0  0.0  0.0  ...   

     V1292  V1293  V1294         V1295  V1296  V129

In [2]:
def getDataset(openmlid):
    ds = openml.datasets.get_dataset(openmlid)
    df = ds.get_data()[0]
    y = df[ds.default_target_attribute].values
    X = pd.get_dummies(df[[c for c in df.columns if c != ds.default_target_attribute]]).values.astype(float)
    return X, y

In [6]:
datasets_suite_271 = [
    1590, 1515, 1457, 1475, 1468, 1486, 1489, 23512, 23517, 4541, 4534, 4538, 4134, 
    4135, 40978, 40996, 41027, 40981, 40982, 40983, 40984, 40701, 40670, 40685, 40900, 
    1111, 42732, 42733, 42734, 40498, 41161, 41162, 41163, 41164, 41165, 41166, 41167, 
    41168, 41169, 41142, 41143, 41144, 41145, 41146, 41147, 41150, 41156, 41157, 41158, 
    41159, 41138, 54, 181, 188, 1461, 1494, 1464, 12, 23, 3, 1487, 40668, 1067, 1049, 
    40975, 31 
]

timeout = 60 * 1

datasets = [1457]#datasets_suite_271
numseeds = 1

pbar = tqdm(total=numseeds * len(datasets))

rows = []
cnt = 0
for openmlid in datasets:
    X, y = getDataset(openmlid)
    for seed in range(numseeds):
        print(openmlid, seed)


        X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, train_size=0.9, random_state=seed)
        print(str(len(X_train)) + " training instances and " + str(len(X_test)) + " test instances.")
        automl = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=timeout)
        automl.fit(X_train, y_train)
        y_hat = automl.predict(X_test)
        score = 1 - sklearn.metrics.accuracy_score(y_test, y_hat)
        print("Accuracy score", score)
        rows.append([openmlid, seed, score])
        pbar.update(1)
pbar.close()
results_sklearn = pd.DataFrame(rows, columns=["openmlid", "seed", "score"])

/home/felix/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


DEBUG:openml.datasets.dataset:Data pickle file already exists and is up to date.


1457 0
1350 training instances and 150 test instances.
[DEBUG] [2021-02-03 02:00:55,285:asyncio] Using selector: EpollSelector
[DEBUG] [2021-02-03 02:00:55,287:asyncio] Using selector: EpollSelector
[INFO] [2021-02-03 02:01:01,921:smac.utils.io.cmd_reader.CMDReader] Output to /tmp/autosklearn_tmp_43ba37b8-65bb-11eb-9aa6-3315768f62dd/smac3-output
[DEBUG] [2021-02-03 02:01:01,922:smac.scenario.scenario.Scenario] SMAC and Scenario Options:
[INFO] [2021-02-03 02:01:01,925:smac.facade.smac_ac_facade.SMAC4AC] Optimizing a deterministic scenario for quality without a tuner timeout - will make SMAC deterministic and only evaluate one configuration per iteration!
[DEBUG] [2021-02-03 02:01:01,926:smac.scenario.scenario.Scenario] Output directory does not exist! Will be created.
[DEBUG] [2021-02-03 02:01:01,927:smac.scenario.scenario.Scenario] Writing scenario-file to /tmp/autosklearn_tmp_43ba37b8-65bb-11eb-9aa6-3315768f62dd/smac3-output/run_1/scenario.txt.
[INFO] [2021-02-03 02:01:02,029:smac.in

In [9]:
inst = sklearn.linear_model.PassiveAggressiveClassifier(C=0.013648712515144728, average=False, fit_intercept = True, loss='squared_hinge', tol = 0.001573909184776253)
X, y = getDataset(1457)
sklearn.model_selection.cross_validate(inst, X, y)["test_score"]

[DEBUG] [2021-02-03 02:11:29,140:openml.datasets.dataset] Data pickle file already exists and is up to date.


array([0.61666667, 0.53666667, 0.63666667, 0.56333333, 0.62666667])

In [20]:
results_sklearn = pd.DataFrame(rows, columns=["openmlid", "seed", "score"])

In [24]:
results_sklearn.groupby("openmlid").agg("mean")

,seed,score
openmlid,,
1111,2.00,0.093103
1457,2.00,0.086207
1468,2.00,0.086207
1475,2.00,0.096552
1486,2.00,0.096552
1489,2.00,0.086207
1515,2.00,0.089655
1590,2.00,0.072414
4134,2.00,0.086207


In [18]:
import numpy as np
np.mean([0.9482758620689655, 0.9310344827586207, 0.9310344827586207, 0.8620689655172413, 0.8620689655172413, 0.8620689655172413, 0.8793103448275862, 0.9310344827586207, 0.8448275862068966, 0.8275862068965517])

0.8879310344827586